In [ ]:
from sklearn.datasets import fetch_openml
boston = fetch_openml(name='boston', version=1, as_frame=True)
df_boston = boston.frame

df_sample = df_boston.sample(n=300, random_state=4713)
print(df_sample.head())

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Estadísticas descriptivas:")
print(df_sample.describe(include="all"))

In [ ]:
corr = df_sample.corr(numeric_only=True)

plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="coolwarm", center=0, annot=True)
plt.title("Matriz de correlación - Muestra Boston (n=300)")
plt.show()

# Variables más correlacionadas con MEDV
print("\nCorrelaciones con MEDV:")
print(corr['MEDV'].sort_values(ascending=False))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cols_plot = ['MEDV','LSTAT','RM','PTRATIO','TAX','CRIM', 'NOX', 'AGE', 'DIS', 'INDUS','B', 'ZN']
n = len(cols_plot)
fig, axes = plt.subplots(4, 3, figsize=(18, 10))  # 4 filas, 3 columnas

for i, c in enumerate(cols_plot):
    if c in df_sample.columns:
        ax = axes[i // 3, i % 3]
        sns.histplot(df_sample[c], bins=30, kde=True, ax=ax)
        ax.set_title(f"Histograma de {c}")

plt.tight_layout()
plt.show()

In [ ]:
top_corr = corr['MEDV'].drop('MEDV').abs().sort_values(ascending=False).head(4).index
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
for i, c in enumerate(top_corr):
    ax = axes[i // 2, i % 2]
    sns.scatterplot(x=df_sample[c], y=df_sample['MEDV'], ax=ax)
    ax.set_xlabel(c)
    ax.set_ylabel('MEDV')
    ax.set_title(f"Dispersión: {c} vs MEDV")

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import statsmodels.api as sm

# 1) Normaliza el nombre de la variable objetivo (MEDV o medv)
target_col = 'MEDV' if 'MEDV' in df_sample.columns else 'medv'

# 2) Asegura que TODO sea numérico y sin NaN
df_num = df_sample.copy()

# Convierte todo a numérico; si hay strings los vuelve NaN
for c in df_num.columns:
    df_num[c] = pd.to_numeric(df_num[c], errors='coerce')

# Elimina filas con NaN (puedes reportar cuántas se fueron)
before = len(df_num)
df_num = df_num.dropna(axis=0).reset_index(drop=True)
print(f"Filas eliminadas por NaN: {before - len(df_num)}")

# 3) Separa X, y (solo columnas numéricas)
y = df_num[target_col].astype(float).values
X = df_num.drop(columns=[target_col]).select_dtypes(include=['number']).astype(float)

# 4) Agrega intercepto
X = sm.add_constant(X, has_constant='add')

# 5) Ajusta GLM Gaussiano
glm_gauss = sm.GLM(y, X, family=sm.families.Gaussian())
res = glm_gauss.fit()
print(res.summary())


In [ ]:
summary_df = res.summary2().tables[1].copy()
sig = summary_df[summary_df['P>|z|'] < 0.05]
print("Variables significativas (p<0.05):")
display(sig[['Coef.', 'Std.Err.', 'P>|z|']])


In [ ]:
ols = sm.OLS(res.model.endog, res.model.exog).fit()
print("\nR²:", round(ols.rsquared, 4))
print("R² ajustado:", round(ols.rsquared_adj, 4))


In [ ]:
residuals = res.resid_response
fitted    = res.fittedvalues

In [ ]:

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,5))
stats.probplot(residuals, dist="norm", plot=ax1)
ax1.set_title("Q-Q plot de residuales")

ax2.scatter(fitted, residuals, alpha=0.7)
ax2.axhline(0, color='red', linestyle='--')
ax2.set_xlabel("Ajustados")
ax2.set_ylabel("Residuales")
ax2.set_title("Residuales vs Ajustados")
plt.tight_layout()
plt.show()